In [8]:
import numpy as np
import random

# Basic Blackjack Strategy Function

### Not using this for the Monte-carlo Evaluation

In [9]:
def BlackJackStrategy(hand, dealer_card):
    hard_count=sum(hand)
    cards=len(hand)
    aces_count= 1*(1 in hand)
    soft_count=hard_count
    good_cards=[1,7,8,9,10]
    
    if cards<2:
        return 'Not enough cards'
    
    if hard_count<=11:
        soft_count= hard_count + 10*aces_count
        
    if cards==2 and soft_count==21:
        return 'Blackjack'
    
    
    if soft_count>=19:
        return 'Stay'
    
    if hand==[9,9] and dealer_card not in [1,7,10]:
        return 'Split'
    
    if hard_count>=17 :
        return 'Stay'

    
    if cards>2:
        if hard_count==soft_count:
            if hard_count<=11:
                return 'Hit'

            if hard_count==12 and dealer_card <=3:
                return 'Hit'

            if dealer_card in good_cards:
                return 'Hit'

            return 'Stay'
        
        
        if soft_count<=17:
            return 'Hit'
        
        if soft_count==18 and dealer_card in [1,9,10]:
            return 'Hit'
        
        
        
        return 'Stay'
    
            
        
    
    if cards==2:
        
        if hand[0]==hand[1]:
            card= hand[0]
            
            if card==1:
                return 'Split'
            
            if card==8 and dealer_card!=1:
                return 'Split'
            
            if card in [2,3,7] and dealer_card not in [1,8,9,10]:
                return 'Split'
            
            if card==6 and dealer_card not in good_cards:
                return 'Split'
            
            if card==4 and dealer_card in [5,6]:
                return 'Split'
            
        if hard_count==soft_count:
            
            if hard_count==11 and dealer_card!=1:
                return 'Double'
            
            if hard_count==10 and dealer_card not in [1,10]:
                return 'Double'
            
            if hard_count==9 and dealer_card not in ([2]+ good_cards):
                return 'Double'
            
            if hard_count<=11:
                return 'Hit'

            if hard_count==12 and dealer_card <=3:
                return 'Hit'

            if dealer_card in good_cards:
                return 'Hit'

            return 'Stay'
            
        
        if hard_count!=soft_count:
            

                
            if soft_count==18:
                
                if dealer_card not in ([2]+good_cards):
                    return 'Double'
                
                if dealer_card in [2,7,8]:
                    return 'Stay'
                
            if soft_count==17 and dealer_card in [3,4,5,6]:
                return 'Double'
            
            if soft_count in [15,16] and dealer_card in [4,5,6]:
                return 'Double'
            
            if soft_count in [13,14] and dealer_card in [5,6]:
                return 'Double'
            
            return 'Hit'
                
                
    
    

In [10]:
BlackJackStrategy([6,5], 1)

'Hit'

# Value Function
3 dimensional array, where first index is \[cardsum(12 to 21) - 12\] (_for indexing_); second index is 0 if no usable ace and 1 if any; third index is dealer card \[(1 to 10)-1\](_for indexing_).

In [68]:
V = np.array([[[0.]*10]*2]*10)
visit_count = np.array([[[0.]*10]*2]*10)

In [54]:
visit_count

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.,

# BlackJack Monte-Carlo Strategy

In [65]:
def pi_20(hand, dealer_card):
    player_sum = sum(hand)
    
    if 1 in hand and player_sum < 12:
            player_sum += 10
    
    if player_sum < 20:
        return 'hit'
    else:
        return 'stick'

In [57]:
def update_values(states, r, V, visit_count):
    for h, a, d in states:
        V[h][a][d] = V[h][a][d] + (1/visit_count[h][a][d])*(r - V[h][a][d]) 

In [1]:
array =([(3.1, 2), (2.1, 4)])

# One Hand Blackjack Simulator

In [63]:
def blackjack_hand_result(V, visit_count):
    #Deck of cards, 4 types of 10
    
    deck=np.array([1,2,3,4,5,6,7,8,9,10,10,10,10])
    
    #Creating a random hand if none is input
    
    player_hand=[random.choice(deck), random.choice(deck)]
        
    dealer_card=random.choice(deck)
    
    # keeps track of all states that the hand has been in during one play
    states = []
    
    player_sum = sum(player_hand)
    
    # an ace that can be used as an 11 i.e. the sum of the cards is less than 12
    usable_ace = 0
    if 1 in player_hand:
        usable_ace = 1
        player_sum += 10
    
    
    if player_sum > 11:
        states.append((player_sum-12, usable_ace, dealer_card-1))
        visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
        
    # keeps track of dealers hand
    dealer_cards=[dealer_card]
    

    # Seeing if player hit blackjack
    if player_sum == 21:

        #Seeing if casino also hit blackjack, in which case tie

        dealer_cards.append(random.choice(deck))
        if 1 in (dealer_cards) and sum(dealer_cards)==11:
            update_values(states, 0, V, visit_count)
            return 0
        
        update_values(states, 1, V, visit_count)
        return 1
    

    else:
    # Seeing how often it says to 'hit'

        while pi_20(player_hand, dealer_card) == 'Hit':
            #Adding one card for every hit
            new_card = random.choice(deck)
            player_hand.append(new_card)
            
            player_sum += new_card
            
            if new_card == 1 and player_sum < 12:
                usable_ace = 1
                player_sum += 10
            
            if usable_ace == 1 and player_sum > 21:
                player_sum -= 10
                usable_ace == 0

            #Player loses bet if hand goes above 21
            if player_sum>21:
                update_values(states, -1, V, visit_count)
                return -1
            
            if player_sum > 11:
                states.append((player_sum-12, usable_ace, dealer_card-1))
                visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
            
    while True:
        #Plays out the blackjack hand from dealer's side

        #Give dealer extra card if loop hasn't broken
        dealer_cards.append(random.choice(deck))


        #Keep track of sum of dealer's cards
        dealer_score= sum(dealer_cards)

        #Keep track of soft score if dealer has an ace

        soft_score= dealer_score
        if dealer_score<=11 and 1 in dealer_cards:
            soft_score+=10

        #If dealer gets blackjack you lose even if you have 21
        if len(dealer_cards)==2 and soft_score==21:
            return -1
        
        #Dealer stays on all 17s
        if soft_score>=17:

            #If dealer bust, player wins bet
            if soft_score>21:
                update_values(states, 1, V, visit_count)
                return 1

            #If player has more than dealer, player wins bet
            if player_sum>soft_score:
                update_values(states, 1, V, visit_count)
                return 1

            #Tie means no money changes hands
            if player_sum==soft_score:
                update_values(states, 0, V, visit_count)
                return 0

            #If player has lower, player loses bet
            if player_sum<soft_score:
                update_values(states, -1, V, visit_count)
                return -1


# Multiple Hand Sim, Test to see house edge in Blackjack

In [29]:
def blackjack_sim(n_hands, V, visit_count):
    for i in range(n_hands):
        blackjack_hand_result(V, visit_count)
     

In [70]:
blackjack_sim(50_000, V, visit_count)
V

array([[[-6.64080068e-01, -2.72727273e-01, -2.65129683e-01,
         -2.30283912e-01, -2.03566122e-01, -2.27972028e-01,
         -4.23423423e-01, -5.35269710e-01, -4.77888730e-01,
         -5.53547309e-01],
        [-7.35301242e-01, -1.73913043e-01, -2.50000000e-01,
         -1.17647059e-01, -2.70270270e-02, -2.42861287e-17,
         -5.90909091e-01, -4.57627119e-01, -4.35897436e-01,
         -6.35529911e-01]],

       [[-7.58184161e-01, -3.11077390e-01, -2.68221574e-01,
         -2.14953271e-01, -9.92366412e-02, -1.30568356e-01,
         -4.51355662e-01, -5.26645768e-01, -5.64648118e-01,
         -5.54226106e-01],
        [-8.13538149e-01, -3.13131313e-01, -1.23595506e-01,
         -2.16494845e-01, -1.90476190e-01, -3.33333333e-01,
         -4.72527473e-01, -4.70588235e-01, -4.41860465e-01,
         -5.56738851e-01]],

       [[-6.28540271e-01, -3.03602058e-01, -2.24699828e-01,
         -1.62251656e-01, -1.38408304e-01, -2.07407407e-01,
         -4.67554077e-01, -4.24703892e-01, -5.04

In [11]:
#Simulates a million hands to estimate house edge
house_edge=-100*(blackjack_sim(n_hands=1_000_000, bet=1) / 1_000_000)
house_edge #percentage

TypeError: blackjack_sim() got an unexpected keyword argument 'bet'

# Strategy Test: Double 7 vs 6

In [10]:
#Player profit from following basic strategy
blackjack_sim(n_hands=100_000, player_h=[4,3], dealer_c=6, bet=1)


2104

In [11]:
blackjack_sim(n_hands=100_000, player_h=[4,3], dealer_c=6, bet=1, hard_c='Double')


-14626

# Strategy Test: Split 10s vs 6

In [12]:
blackjack_sim(n_hands=100_000, player_h=[10,10], dealer_c=6, bet=1)


70626

In [13]:
blackjack_sim(n_hands=100_000, player_h=[10,10], dealer_c=6, bet=1, hard_c='Split')


60517.5